### calculate radial profile

In [1]:
import  numpy               as     np
from    scripts.tools       import *
from    astropy.table       import Table
from    astropy.io          import fits
from    astropy.cosmology   import Planck18
import  astropy.units       as     u
import  matplotlib.pyplot   as     plt
import  matplotlib.colors   as     colors  
from    matplotlib          import use
from    tqdm.notebook       import tqdm
from    concurrent.futures  import ThreadPoolExecutor, as_completed
import  sys, os
from    IPython.display     import clear_output
from    astropy.wcs         import WCS
from    photutils.aperture  import EllipticalAnnulus, EllipticalAperture, aperture_photometry
obj_lis = Table.read('obj_lis_selected.fits')


In [8]:
import warnings

def K_lambda(line='Ha'):
    #calzetti_attenuation
    """
    Calculate the dust attenuation value k(lambda) from the Calzetti et al. (2000) attenuation curve.
    Parameters:
        wavelength_nm (float): Wavelength in nanometers (nm).
    """
    wavelength_um =  (0.6563 if line == 'Ha' else 0.4861)
    if 0.12 <= wavelength_um <= 0.63:
        # Formula for the UV to optical wavelength range
        k_lambda = 2.659 * (-1.857 + 1.040 / wavelength_um) + 4.05
    elif wavelength_um > 0.63:
        # Formula for the near-infrared wavelength range
        k_lambda = 2.659 * (-2.156 + 1.509 / wavelength_um - 0.198 / (wavelength_um ** 2) + 0.011 / (wavelength_um ** 3)) + 4.05
    return k_lambda


def radial_profile(obj, linemap, weight, seg, pixel_length):
    # Here we try to use elliptical annuli to extract the radial profile
    # The semi major axis in arcsec, transformed to pixel:
    semi_major = obj['re'] / 0,1 * pixel_length
    # The axis ratio
    axis_ratio = obj['q']
    # The position angle
    pa = obj['pa']
    # The center of the object
    center = (linemap.data.shape[0] / 2, linemap.data.shape[1] / 2)
    # A series of elliptical annuli with the same width
    r = np.linspace(0, linemap.data.shape[0]/2, int(linemap.data.shape[0])+1)
    # The elliptical annuli
    center_annuli = [EllipticalAperture(center, r[1], r[1] * axis_ratio, theta=pa * u.deg.to(u.rad))]
    ellip_annuli = [EllipticalAnnulus(center, a_in=r[i], a_out=r[i+1], b_in=r[i] * axis_ratio, b_out=r[i+1] * axis_ratio, theta=pa * u.deg.to(u.rad)) for i in range(1, len(r)-1)]
    
    final_aperture = center_annuli + ellip_annuli
    # Calculate the area of each annulus
    surface_area = [aperture.area for aperture in final_aperture]

    # Initialize arrays to store the results
    ha_r = np.zeros(len(final_aperture))
    ha_r_err = np.zeros(len(final_aperture))
    
    # Mask to select the object
    mask = np.logical_or(seg != obj['ID'],linemap.data<=0)
    

    error_data = np.where(weight.data**0.5 > 0, 1 / weight.data**0.5, np.nan)
    error_data[np.isinf(error_data)] = np.nan

    # Loop over each annulus and calculate the surface brightness
    for i, annulus in enumerate(final_aperture):
        phot_table = aperture_photometry(linemap.data, annulus, error=error_data, mask=mask, method='subpixel', subpixels=50)
        ha_r[i] = phot_table['aperture_sum'][0]/surface_area[i]
        ha_r_err[i] = phot_table['aperture_sum_err'][0]    
    return r*pixel_length, ha_r, ha_r_err

#this function will generate the             print(r, ha_r, ha_r_err, hb_r, hb_r_err, balmer_r, balmer_r_err)
#radial table for a given object
def gen_radial_table_ellip(obj,
                     LINE_HA='LINE_HA',      LINE_HB='LINE_HB_CONV',
                     LINEWHT_HA='LINEWHT_HA',LINEWHT_HB='LINEWHT_HB_CONV'):
    try:
        path = f"data_extracted/{file_name(obj,prefix='extracted')}"
        with fits.open(path,mode='update') as hdu:
            if find_data('SEG_MOD',hdu) != None:
                seg_map = find_data('SEG_MOD',hdu)[1].data
            else:
                seg_map = find_data('SEG',hdu)[1].data

            #extract the radial profile surface brightness
            r, ha_r, ha_r_err = radial_profile(obj,
                                            linemap      = find_data(LINE_HA,hdu)[1],
                                            weight       = find_data(LINEWHT_HA,hdu)[1],
                                            seg          = seg_map,
                                            pixel_length = obj['pixel_length'])
            
            r, hb_r, hb_r_err = radial_profile(obj,
                                            linemap      = find_data(LINE_HB,hdu)[1],
                                            weight       = find_data(LINEWHT_HB,hdu)[1],
                                            seg          = seg_map,
                                            pixel_length = obj['pixel_length'])

            r, balmer_r_pix, balmer_r_pix_err = radial_profile(obj,
                                            linemap      = find_data('2D_BALMER',hdu)[1],
                                            weight       = find_data('2D_BALMER_ERR',hdu)[1],
                                            seg          = seg_map,
                                            pixel_length = obj['pixel_length'])

            #calculate the balmer decrement
            balmer_r     = ha_r/hb_r
            balmer_r_err = ((ha_r_err/hb_r)**2 + (hb_r_err**2 * (ha_r/hb_r**2)**2))**0.5
            
            #now calculate the extinction
            #color excess
            E_ba = 2.5*np.log10(balmer_r/2.86)
            #attenutation
            A_ba = E_ba/(K_lambda('Hb')-K_lambda('Ha')) * K_lambda('Ha')


            E_pix = 2.5*np.log10(balmer_r_pix/2.86)
            A_pix = E_pix/(K_lambda('Hb')-K_lambda('Ha')) * K_lambda('Ha')

            #columns for the radial table
            cols = [
                fits.Column(name='DISTANCE [kpc]',                       format='E', array=r),
                fits.Column(name='Ha_SURF_BRIGHT [1e-17 erg/s/cm2]',     format='E', array=ha_r),
                fits.Column(name='Ha_SURF_BRIGHT_err [1e-17 erg/s/cm2]', format='E', array=ha_r_err),
                fits.Column(name='Hb_SURF_BRIGHT [1e-17 erg/s/cm2]',     format='E', array=hb_r),
                fits.Column(name='Hb_SURF_BRIGHT_err [1e-17 erg/s/cm2]', format='E', array=hb_r_err),
                fits.Column(name='BALMER_DECREM',                        format='E', array=balmer_r),
                fits.Column(name='BALMER_DECREM_ERR',                    format='E',array=balmer_r_err),
                fits.Column(name='E_BV',                                  format='E', array=E_ba),
                fits.Column(name='A_Ha',                                  format='E', array=A_ba)
            ]

            cols_pix = [
                fits.Column(name='DISTANCE [kpc]',                       format='E', array=r),
                fits.Column(name='Ha_SURF_BRIGHT [1e-17 erg/s/cm2]',     format='E', array=ha_r),
                fits.Column(name='Ha_SURF_BRIGHT_err [1e-17 erg/s/cm2]', format='E', array=ha_r_err),
                fits.Column(name='Hb_SURF_BRIGHT [1e-17 erg/s/cm2]',     format='E', array=hb_r),
                fits.Column(name='Hb_SURF_BRIGHT_err [1e-17 erg/s/cm2]', format='E', array=hb_r_err),
                fits.Column(name='BALMER_DECREM',                        format='E', array=balmer_r_pix),
                fits.Column(name='BALMER_DECREM_ERR',                    format='E',array=balmer_r_pix_err),
                fits.Column(name='E_BV',                                  format='E', array=E_pix),
                fits.Column(name='A_Ha',                                  format='E', array=A_pix)
            ]
            
            #choose the right name for saving the radial table
            if 'CONV' in LINE_HB:
                name_addon = '_CONV'
            else:
                name_addon = ''
            if 'BG' not in LINE_HA:
                name = f'RAD_PROFILE_ellip_{name_addon}'
                new_table = fits.BinTableHDU.from_columns(cols, name=name)
                new_table_pix = fits.BinTableHDU.from_columns(cols_pix, name=name+'_PIX')
            else:
                name = f'RAD_PROFILE_ellip{name_addon}_BG'
                new_table = fits.BinTableHDU.from_columns(cols, name=name)
                new_table_pix = fits.BinTableHDU.from_columns(cols_pix, name=name+'_PIX')



            #save or update table
            save_update(new_table,hdu)
            save_update(new_table_pix,hdu)
            hdu.flush()
            return f"{obj['subfield']}-{obj['ID']} processed"
    except Exception as e:
            return f"! {obj['subfield']}-{obj['ID']} failed, error{e}"

def cat_process(obj_lis,
                LINE_HA,   LINE_HB,
                LINEWHT_HA, LINEWHT_HB,
                max_threads=1):
        print(f'start process,{LINE_HA},{LINE_HB}')
        results = []
        if max_threads > 1 :
            with ThreadPoolExecutor(max_threads) as executor:
                futures = {executor.submit(
                    gen_radial_table_ellip,
                    obj,LINE_HA,LINE_HB,LINEWHT_HA,LINEWHT_HB
                                            ): obj for obj in obj_lis}
                for future in tqdm(as_completed(futures), total=len(obj_lis), desc="Processing"):
                    results.append(future.result())
            return results
        else:
            for obj in tqdm(obj_lis):
                results.append(gen_radial_table_ellip(obj,LINE_HA,LINE_HB,LINEWHT_HA,LINEWHT_HB))
            return results


def main():
    warnings.filterwarnings('ignore', category=RuntimeWarning)
    obj_lis = Table.read('obj_lis_selected.fits')
    '''
    results1 = cat_process(obj_lis,
                        LINE_HA='LINE_HA',LINE_HB='LINE_HB',
                        LINEWHT_HA='LINEWHT_HA',LINEWHT_HB='LINEWHT_HB',max_threads=1)
    errorcounting(results1)
    '''
    results3 = cat_process(obj_lis,
                        LINE_HA='LINE_HA',LINE_HB='LINE_HB_CONV',
                        LINEWHT_HA='LINEWHT_HA',LINEWHT_HB='LINEWHT_HB_CONV',max_threads=1)
    errorcounting(results3)

'''
    results2 = cat_process(obj_lis,
                        LINE_HA='LINE_HA_BG',LINE_HB='LINE_HB_BG',
                        LINEWHT_HA='LINEWHT_HA',LINEWHT_HB='LINEWHT_HB',max_threads=6)
    errorcounting(results2)
    
    results4 = cat_process(obj_lis,
                        LINE_HA='LINE_HA_BG',LINE_HB='LINE_HB_CONV_BG',
                        LINEWHT_HA='LINEWHT_HA',LINEWHT_HB='LINEWHT_HB_CONV',max_threads=6)
    errorcounting(results4)
'''
if __name__ == '__main__':
    main()


start process,LINE_HA,LINE_HB_CONV


  0%|          | 0/158 [00:00<?, ?it/s]

total number of obj processed: 158
number of failed obj 5
